In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='eb031d6a-7903-44c8-9bff-b83a07e02b3e', project_access_token='p-129dec77760a1b06bac077a4278a8e9021bcf216')
pc = project.project_context


In [ ]:
import datetime 

import IPython.display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.linear_model
import sklearn.model_selection

In [ ]:
pip install pandas_datareader

In [ ]:
import pandas_datareader

In [ ]:
# DWH
df_ibm = pandas_datareader.data.DataReader('IBM',  'yahoo', '2018-01-01')
df_amzn = pandas_datareader.data.DataReader('AMZN',  'yahoo', '2018-01-01')
df_msft = pandas_datareader.data.DataReader('MSFT',  'yahoo', '2018-01-01')

In [ ]:
df_ibm.tail(3)

In [ ]:
project.save_data("stock_ibm.csv", df_ibm.to_csv(), overwrite=True)

In [ ]:
df_amzn.tail(3)

In [ ]:
project.save_data("stock_amzn.csv", df_amzn.to_csv(), overwrite=True)

In [ ]:
df_msft.tail(3)

In [ ]:
project.save_data("stock_msft.csv", df_msft.to_csv(), overwrite=True)

In [ ]:
# Simple Moving Average
df_ibm['SMA'] = df_ibm['Close'].rolling(window=14).mean()
df_ibm['Close'].plot(figsize=(15,6), color="red")
df_ibm['SMA'].plot(figsize=(15,6), color="green")
plt.show()

In [ ]:
# Data mining
df_ibm['change'] = (((df_ibm['Close'] - df_ibm['Open'])) / (df_ibm['Open']) * 100)
df_amzn['change'] = (((df_amzn['Close'] - df_amzn['Open'])) / (df_amzn['Open']) * 100)
df_msft['change'] = (((df_msft['Close'] - df_msft['Open'])) / (df_msft['Open']) * 100)
df_ibm.tail(2).round(2)

In [ ]:
df_ibm['Close'].plot(figsize=(15,6), color="blue")
df_amzn['Close'].plot(figsize=(15,6), color="orange")
plt.show()

In [ ]:
df_ibm['Close'].plot(figsize=(15,6), color="blue")
df_msft['Close'].plot(figsize=(15,6), color="green")
plt.show()

In [ ]:
df_ibm['change'].tail(100).plot(grid=True, figsize=(15,6), color="blue")
df_amzn['change'].tail(100).plot(grid=True, figsize=(15,6), color="orange")
df_msft['change'].tail(100).plot(grid=True, figsize=(15,6), color="green")
plt.show()

In [ ]:
# Machine learning
df_ibm['label'] = df_ibm['Close'].shift(-30)
df_ibm.tail(40)

In [ ]:
X = np.array(df_ibm.drop(['label', 'SMA'], axis=1))
X = sklearn.preprocessing.scale(X)

predict_data = X[-30:]
X = X[:-30]
y = np.array(df_ibm['label'])
y = y[:-30]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size = 0.2)

lr = sklearn.linear_model.LinearRegression()
lr.fit(X_train,y_train)

accuracy = lr.score(X_test, y_test)
accuracy

In [ ]:
predicted_data = lr.predict(predict_data)
predicted_data

In [ ]:
df_ibm['Predict'] = np.nan

last_date = df_ibm.iloc[-1].name

one_day = 86400
next_unix = last_date.timestamp() + one_day

for data in predicted_data:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df_ibm.loc[next_date] = np.append([np.nan]* (len(df_ibm.columns)-1), data)

df_ibm['Close'].plot(figsize=(15,6), color="blue")
df_ibm['Predict'].plot(figsize=(15,6), color="pink")
plt.show()

In [ ]:
project.save_data("stock_ibm_future.csv", df_ibm['Predict'].to_csv(), overwrite=True)

In [ ]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

# stock price change forecast
d = df_ibm['Predict'].tail(30)[0] - df_ibm['Close'].tail(31)[0]
print(Decimal(str(d)).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))

In [ ]:
# stock change rate forecast
c = ((df_ibm['Predict'].tail(30)[0] - df_ibm['Close'].tail(31)[0]) / df_ibm['Close'].tail(31)[0]) *100
print(Decimal(str(c)).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))

In [ ]:
if c > 0:
    UpDown="Up"
else:
    UpDown="Down"
print(UpDown)

In [ ]:
# send stock forecast of the next date to slack
import requests
import json
import termcolor

post_url="<Slack Webhook URL>"

#difine some data
if c > 0:
    UpDown=":arrow_double_up:"
else:
    UpDown=":arrow_double_down:"

notification="Hi, <username>."+"\n\n"+":ibm: stock forecast for tomorrow:"+"\n"+"[ "+str(round(df_ibm['Predict'].tail(30)[0], 2))+" ]"+"\n\n"+"Oops! Change"+UpDown+"  "+str(round(d, 2))+" / "+str(round(c, 2))+"%"+"\n\n"+"Have a nice day :woman-cartwheeling:"

def post_slack(content):
    requests.post(
        post_url,
        data=json.dumps(
            {"text": content,
             "username": "StockForecast",
             "icon_emoji": ":wat:"}))

post_slack(notification)